In [2]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [37]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=19)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [50]:
from cluster_model import find_max_weight

find_max_weight(decomp.lattice_scores, K=20)

Most negative absolute value: 1e-06
We have 120 clusters
[(3465002, [0, 8, 57, 89, 120])]


(None, <networkx.classes.digraph.DiGraph at 0x71990df428b0>)

In [ ]:
max_score = max(max(d) for d in decomp.lattice_scores)
min_score = min(min(d) for d in decomp.lattice_scores)
lattice_scores_as_probs = [[(d - min_score) / (max_score - min_score)
                            for d in ds] for ds in decomp.lattice_scores]


def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"] *
                              trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"]*trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    return V

In [46]:
score_path= [8, 57, 89]
score_path= [10, 63, 89]
to_score = [d[:100] for d in ds][:10]
scores = decomp.score(
	to_score,
	score_path=score_path
)

DS (3, 278, 512) 278
(278, 3, 512) [10, 63, 89]


In [47]:
tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)['input_ids']] for d in to_score]

In [48]:
from circuitsvis.utils.render import RenderedHTML, render

# TODO sep fun
html = render(
        "TextNeuronActivations",
        tokens=tokens,
        activations=[[[[tok]] for tok in s] for s in scores],
        firstDimensionName="Layer",
        secondDimensionName="Neuron",
        firstDimensionLabels=None,
				secondDimensionLabels=None
    )

In [49]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_319930/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
